In [ ]:
import urllib.request
import os
from dotenv import load_dotenv
import json
from io import StringIO

# 환경 변수 로딩
load_dotenv()
service_key = os.getenv("GBUS_API_KEY")

# Step 1. 기반정보 API 호출
url = f"http://apis.data.go.kr/6410000/baseinfoservice/v2/getBaseInfoItemv2?format=json&serviceKey={service_key}"
req = urllib.request.Request(url)
with urllib.request.urlopen(req) as res:
    data = res.read().decode('utf-8')

    parsed = json.loads(data)
    route_file_url = parsed['response']['msgBody']['baseInfoItem']['routeDownloadUrl']

# Step 2. 노선 데이터 파일 다운로드
route_txt = urllib.request.urlopen(route_file_url).read().decode('utf-8')

# 저장 경로 지정
output_path = os.path.join("output", "route20250424V2.txt")

# 파일로 저장
os.makedirs("output", exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    f.write(route_txt)

print(f"✅ 텍스트 파일 저장 완료: {output_path}")

📄 응답 내용 미리보기:
 {"response":{"comMsgHeader":"","msgHeader":{"queryTime":"2025-04-24 23:57:31.765","resultCode":0,"resultMessage":"정상적으로 처리되었습니다."},"msgBody":{"baseInfoItem":{"areaDownloadUrl":"http:\/\/openapi.gbis.go.kr\/ws\/download?area20250424V2.txt","areaVersion":20250424,"routeDownloadUrl":"http:\/\/openapi.gbis.go.kr\/ws\/download?route20250424V2.txt","routeLineDownloadUrl":"http:\/\/openapi.gbis.go.kr\/ws\/download?routeline20250424V2.txt","routeLineVersion":20250424,"routeStationDownloadUrl":"http:\/\/
📥 routeDownloadUrl: http://openapi.gbis.go.kr/ws/download?route20250424V2.txt
✅ 텍스트 파일 저장 완료: output\route20250424V2.txt


In [3]:
import pandas as pd
import json

# Step 1. 텍스트 읽기
with open('output/route20250424V2.txt', encoding='utf-8') as f:
    raw = f.read()

# Step 2. ^ 기준으로 레코드 나누기 (헤더 + 데이터)
records = raw.strip().split('^')

# Step 3. 헤더 + 상위 5개 테스트만 하려면
# records = records[:6]  # 이건 테스트용일 때만!

# Step 4. 헤더와 본문 분리
columns = records[0].split('|')
rows = [r.split('|') for r in records[1:] if r.strip()]

# Step 5. DataFrame 생성
df = pd.DataFrame(rows, columns=columns)

# Step 6. 문자열 기준 필터링 (중요!)
target_codes = ['11', '14']
df_filtered = df[df['routeTypeCd'].isin(target_codes)]
print(len(df_filtered))

# Step 7. 필요한 컬럼만 추출하여 리스트로 변환
result = df_filtered[['routeId', 'routeName']].values.tolist()

# Step 8. JSON 저장
with open('output/crawlering_route_ids.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"✅ 저장 완료! 추출된 노선 수: {len(result)}개")


399
✅ 저장 완료! 추출된 노선 수: 399개


In [1]:
import json

# 1. 원본 파일 열기
with open("temp\경기도_BMS정류소정보.json", "r", encoding="utf-8") as f:
    station_data = json.load(f)

# 2. 필요한 필드만 추출해서 딕셔너리 생성
filtered_station_dict = {
    station["STTN_ID"]: station["STTN_NM"]
    for station in station_data
    if "STTN_ID" in station and "STTN_NM" in station
}

# 3. 새 파일로 저장
with open("temp/station.json", "w", encoding="utf-8") as f:
    json.dump(filtered_station_dict, f, ensure_ascii=False, indent=4)

print("✅ station.json 생성 완료!")

✅ station.json 생성 완료!
